# 3. 基于Quingo的首个量子程序提交

如果需要在云平台量子计算机上使用其他量子语言提交实验，思路如下：首先利用所选编程语言，在用户本地调用语言编译器，将线路编译/输出为QCIS语言，然后按照QCIS语言进行实验提交。

Quingo中集成了与云平台的对接功能，用户无需手动提交到云平台，直接使用quingo的接口即可。

（云平台JupyterLab环境已经预制好多种语言环境并且都无需安装，如有其他需求，可以联系客服。）

## 3.1 Quingo环境的安装与配置

Quingo可以通过以下两种方式安装：

（当前Quingo的最新版本为0.2.1）

### 3.1.1 源码安装

In [ ]:
git clone git@gitee.com:quingo/quingo-runtime.git
cd quingo-runtime
git checkout develop
pip install . -e
python -m quingo.install_quingoc

### 3.1.2 pip安装

In [ ]:
pip install quingo
python -m quingo.install_quingoc

## 3.2 编写Quingo量子程序

根据Quingo语法规则，编写Quingo量子程序。

Quingo量子程序通常以`.qu`为文件后缀，并保存在Python主程序所在目录。
用户可以使用vscode中的语法高亮插件`vscode-quingo`来辅助代码编写。

下面的`bell.qu`是一个Quingo的例子：

用户也可以直接使用Python将Quingo代码写入文件：

In [1]:
from pathlib import Path

Quingo_code='''
import std_ops

operation bell_state() : unit {
    // 指定物理量子比特，逻辑比特q0对应物理比特q0，逻辑比特q1对应物理比特q6
    using(q0: qubit = 0, q1: qubit = 6) {
        H(q0);
        X(q1);
        CNOT(q0, q1);
        measure(q0);
        measure(q1);
    }
}
'''

file_path = Path("./bell.qu")
file_path.write_text(Quingo_code)

239

## 3.3 在量子计算云平台上运行Quingo程序

对云平台的访问和任务提交已集成到Quingo库中，用户可直接调用接口，在云平台上运行Quingo程序。

下面是一个完整的例子，展示了Quingo程序从编译到执行的全过程。

In [2]:
from quingo import *
from pathlib import Path

qu_file = Path("./bell.qu")

def routine(circ_name, num_shots=1):
    # 指定quingo文件和quingo主函数名
    task = Quingo_task(qu_file, circ_name)
    # 指定执行控制信息
    cfg = ExeConfig(
        ExeMode.RealMachine,   # 执行模式，云平台上的执行均为RealMachine模式
        num_shots,   # 单个线路的执行次数
        xh_login_key="7e6999bab11453428b8ded1fac00b3ea",    # 用户SDK密钥，需要替换为用户自己的密钥
        xh_machine_name="Xiaohong",    # 量子计算机
    )
    # 将quingo程序编译为qcis程序
    qasm_fn = compile(task, params=(), qasm_fn="./bell_state.qcis")
    # 将qcis程序提交到云平台并执行
    res = execute(qasm_fn, BackendType.XIAOHONG, cfg)
    print("result: ", res)

routine("bell_state", 10)

Set account successfully:
   login key = 7e699***************************
   machine name = Xiaohong
Start execute:
num shots = 10
查询实验结果请等待: 5.81秒
查询实验结果请等待: 4.24秒
查询实验结果请等待: 4.18秒
result:  {'qubits': [0, 6], 'results': [[0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 0], [0, 1], [1, 0]]}


## 3.4 （可选）编译程序，输出QCIS线路

Quingo文件经过编译会生成QCIS文件，用户在调试程序时，查看QCIS文件可能有所帮助。

读取Quingo编译输出的QCIS文件：

In [3]:
f = open("./bell_state.qcis",'r')
quingo_qcis=f.read()
f.close()
print(quingo_qcis)

    H    Q0
    X    Q6
    H    Q6
   CZ    Q0           Q6
    H    Q6
    M    Q0
    M    Q6

